# 1.前期准备

## 1.1载入各种机器学习及数据科学库

In [7]:
# 导入所需要的库
import pandas as pd # 用于处理数据的工具
import lightgbm as lgb # 机器学习模型 LightGBM
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error # 评分 MAE 的计算函数
from sklearn.model_selection import train_test_split # 拆分训练集与验证集工具
from tqdm import tqdm # 显示循环的进度条工具
import warnings # 忽略警告信息
warnings.filterwarnings("ignore")#在接下来的代码执行过程中忽略所有的警告信息，不再显示在控制台或日志中。


## 1.2 读取数据和数据准备

In [4]:
# 数据准备
train_dataset = pd.read_csv("C://Users//qiao//Downloads//train.csv") # 原始训练数据。
test_dataset = pd.read_csv("C://Users//qiao//Downloads//test.csv") # 原始测试数据（用于提交）。

submit = pd.DataFrame() # 定义提交的最终数据。
submit["序号"] = test_dataset["序号"] # 对齐测试数据的序号。

MAE_scores = dict() # 定义评分项。

# 2.建立模型

## 2.1 设定参数

In [5]:
# 参数设置
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。
train_set, valid_set = train_test_split(train_dataset, test_size=0.2) # 拆分数据集。

# 设定 LightGBM 训练参
lgb_params = {
        'boosting_type': 'gbdt', # 指定梯度提升树的类型，'gbdt'表示传统的梯度提升树。
        'objective': 'regression', # 指定模型的优化目标，这里是回归任务，因此使用'regression'表示回归问题。
        'metric': 'mae', # 在训练过程中评估模型性能的指标，这里使用'MAE'（平均绝对误差）作为回归模型的评估指标。
        'min_child_weight': 5, # 指定每个叶子节点最小的样本权重和（叶子节点的最小样本数量），用于防止过拟合。
        'num_leaves': 2 ** 5, # 指定每棵树上的叶子节点数，这里是32个叶子节点。
        'lambda_l2': 10, # L2正则化的惩罚系数，用于控制模型的复杂度。
        'feature_fraction': 0.8, #  指定训练每棵树时使用的特征的比例，用于加速训练和防止过拟合。
        'bagging_fraction': 0.8, # 指定训练每棵树时使用的样本的比例，用于加速训练和防止过拟合。
        'bagging_freq': 4, #  指定bagging的频率，也就是执行多少次bagging，用于加速训练。
        'learning_rate': 0.05, # 指定梯度下降步长的学习率，用于控制每次迭代参数更新的幅度。
        'seed': 2023, # 设置随机种子，用于保证实验的可重复性。
        'nthread' : 16, # 指定并行线程数，用于加速训练过程。
        'verbose' : -1, # 控制训练过程中输出信息的级别。-1:输出任何信息，0:输出少量信息，大于0:输出更详细的信息。
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。

## 2.2提取特征

In [14]:
# 时间特征函数
def time_feature(data: pd.DataFrame, pred_labels: list=None) -> pd.DataFrame: # 提取数据中的时间特征。
        
    data = data.copy() # 复制数据，避免后续影响原始数据。
    data = data.drop(columns=["序号"]) # 去掉”序号“特征。
    
    data["时间"] = pd.to_datetime(data["时间"]) 
    # 将”时间“特征的文本内容转换为 Pandas 可处理的格式。  
    data["month"] = data["时间"].dt.month 
    # 添加新特征“month”，代表”当前月份“。
    data["day"] = data["时间"].dt.day 
    # 添加新特征“day”，代表”当前日期“。
    data["hour"] = data["时间"].dt.hour 
    # 添加新特征“hour”，代表”当前小时“。
    data["minute"] = data["时间"].dt.minute 
    # 添加新特征“minute”，代表”当前分钟“。
    data["weekofyear"] = data["时间"].dt.isocalendar().week.astype(int) 
    # 添加新特征“weekofyear”，代表”当年第几周“，并转换成 int，否则 LightGBM 无法处理。
    data["dayofyear"] = data["时间"].dt.dayofyear 
    # 添加新特征“dayofyear”，代表”当年第几日“。
    data["dayofweek"] = data["时间"].dt.dayofweek 
    # 添加新特征“dayofweek”，代表”当周第几日“。
    data["is_weekend"] = data["时间"].dt.dayofweek // 6 
    # 添加新特征“is_weekend”，代表”是否是周末“，1 代表是周末，0 代表不是周末。

    data = data.drop(columns=["时间"]) 
    # LightGBM 无法处理这个特征，它已体现在其他特征中，故丢弃。

    if pred_labels: # 如果提供了 pred_labels 参数，则执行该代码块。
        data = data.drop(columns=[*pred_labels]) # 去掉所有待预测的标签。
    
    return data # 返回最后处理的数据，其中包含新生成的时间特征和其他特征。

test_features = time_feature(test_dataset) 
# 调用该函数处理测试集的时间特征，无需 pred_labels。并将结果存储在名为test_features的DataFrame中。

test_features.head(5)

,流量1,流量2,流量3,流量4,流量5,流量6,流量7,流量8,流量9,流量10,...,下部温度设定16,下部温度设定17,month,day,hour,minute,weekofyear,dayofyear,dayofweek,is_weekend
0,24.222000,24.070000,28.889000,25.264999,26.827999,4.354,21.784,24.323000,16.721001,18.652000,...,837,750,3,1,4,15,9,60,2,0
1,24.138000,23.974001,28.798000,25.517000,27.108999,4.377,21.886,24.384001,16.709999,18.600000,...,837,750,3,1,4,20,9,60,2,0
2,24.152000,24.006001,28.827999,25.114000,26.719999,4.293,21.611,24.099001,16.591999,18.471001,...,837,750,3,1,4,26,9,60,2,0
3,24.108999,23.974001,28.783001,25.114000,27.047001,4.354,21.857,24.337999,16.677999,18.561001,...,837,750,3,1,4,31,9,60,2,0
4,24.350000,24.215000,29.146999,25.416000,25.416000,4.377,21.900,24.444000,16.775999,18.704000,...,837,750,3,1,4,36,9,60,2,0


# 3 训练与拟合

## 3.1 开始训练

In [22]:
 
# 从所有待预测特征中依次取出标签进行训练与预测。
for pred_label in tqdm(pred_labels): #通过for循环，依次遍历所有的待预测特征（标签）。
   
    train_features = time_feature(train_set, pred_labels=pred_labels) # 处理训练集的时间特征。
   
    train_labels = train_set[pred_label] # 训练集的标签数据。
    
    train_data = lgb.Dataset(train_features, label=train_labels) # 将训练集转换为 LightGBM 可处理的类型。

    valid_features = time_feature(valid_set, pred_labels=pred_labels) # 处理验证集的时间特征。
    
    valid_labels = valid_set[pred_label] # 验证集的标签数据。
    
    valid_data = lgb.Dataset(valid_features, label=valid_labels) # 将验证集转换为 LightGBM 可处理的类型。
    
    
    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（750）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 750, valid_sets=valid_data, callbacks=[no_info])
    
    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration) 
    # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) 
    # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, valid_labels) 
    # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score 
    # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。
     
submit.to_csv('submit_result_new.csv', index=False) # 保存最后的预测结果到 submit_result.csv

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:37<00:00,  1.11s/it]


这段代码是一个循环，用于逐个从所有待预测特征中取出标签，然后进行模型训练和预测的过程。以下是主要的步骤：

1. 循环迭代：通过for循环，依次遍历所有的待预测特征（标签）。

2. 数据处理：对训练集（train_set）和验证集（valid_set）分别进行时间特征的提取和预处理，利用`time_feature`函数来处理数据。

3. 构建数据集：将处理后的训练集和验证集转换为LightGBM可处理的数据集类型，分别命名为`train_data`和`valid_data`。

4. 训练模型：使用LightGBM进行模型训练，根据给定的参数`lgb_params`进行训练，并设定模型的迭代次数为200次。模型在训练过程中会使用验证集来监测模型的性能。

5. 预测：使用训练好的模型对验证集和测试集（由`test_features`表示）进行预测。

6. 评估模型性能：计算验证集预测数据与真实标签之间的均方绝对误差（MAE），并将对应标签的MAE值存储在字典`MAE_scores`中。

7. 结果保存：将测试集的预测结果存入最终提交数据`submit`中，并将最终提交数据保存到名为`submit_result.csv`的CSV文件中。

该代码段最终将对每个待预测特征（标签）训练模型、进行预测，并将最终结果保存到CSV文件中，用于提交或后续的进一步分析。

In [24]:
MAE_scores# 查看各项的 MAE 值。

{'上部温度1': 3.6151018577992575,
 '上部温度2': 3.418981764862061,
 '上部温度3': 2.522942261137115,
 '上部温度4': 1.5927955876957651,
 '上部温度5': 2.062921703361124,
 '上部温度6': 1.9077029430178185,
 '上部温度7': 1.197775772810529,
 '上部温度8': 0.15080553641339703,
 '上部温度9': 0.0763939825278096,
 '上部温度10': 0.06530821641357723,
 '上部温度11': 0.09243099486012629,
 '上部温度12': 0.10271846262577626,
 '上部温度13': 0.09243354917515438,
 '上部温度14': 0.13637568883246176,
 '上部温度15': 0.23530386815034743,
 '上部温度16': 0.2985791753615057,
 '上部温度17': 0.8462028297405895,
 '下部温度1': 2.412943081589551,
 '下部温度2': 1.3333420061515278,
 '下部温度3': 0.6094141211573907,
 '下部温度4': 0.3552909991372161,
 '下部温度5': 0.8206960556434971,
 '下部温度6': 0.5768786177629707,
 '下部温度7': 0.2906605247221228,
 '下部温度8': 0.07690291653319886,
 '下部温度9': 11.565785244929733,
 '下部温度10': 0.07110735178918888,
 '下部温度11': 0.1215781607702465,
 '下部温度12': 0.12860161565854714,
 '下部温度13': 0.10636059564290262,
 '下部温度14': 0.12331410195754595,
 '下部温度15': 0.1948635875490879,
 '下部温度16': 0.246496